# Bayestar Code for followup 

The below code is composed of two pieces 

1. do_bayestar_run
Do bayestar run performs takes in ra/dec in radians in the range (-pi, pi), (-pi/2, pi/2), and then performs a byaestar run on that locaiton. It begins by using the OS module to call a lalapps_inspinj command that generates a parameter space given your inputs. Then, bayestar_coincs is called to build the space of waveforms off of the space of parameters you previously specified.  Bayestar_realize_coincs is then called to perform match filtering throughout the given space of waveforms, building the data needed to form a sky localization region for the waveforms. The last calls then create an image plot that dispalys the localization region

NOTE: to run this code, you need to have your current directory send to a dir which contains the 'correct_psd.xml' file which ghosh was able to get his hands on


2. return_percentile_index
Given a set of healpy data and a percentile, this code finds the pixel which corresponds to the percentile; this 'correspondence' is found by first linearing ordering the values of the healpy data, and summing over the pixels until the sum equals the sepcified percentile value. The pixel this corresponds to is then reutrned.


outisde of that, there is one more block of code which (A) takes in some pre-genereated fits files by the 'do_bayestar_run' code (B) generates the healpy data corresponding to it (C) gets the specified percentile pixel through the 'return_percentile_index' code, and (d) returns the correct right ascension and declination positions corresponding to that pixel. The RA/DEC values are returned in degrees in bayestar format (e.g.  (-180, 180), (-90, 90)) . To convert this format back to the bilby format, we have to add 180 degrees to the right ascension value. 


In [47]:
#import np/matplotlin/healpy/os/bilby/astropy stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import healpy as hp
import os
import bilby as bby
from astropy.io import fits
import json

#function to read in skymap localization regions from fits files  
from ligo.skymap.io.fits import read_sky_map
from ligo.skymap.tool import ArgumentParser, FileType, register_to_xmldoc

#angular distance function
from PyAstronomy import pyasl

import argparse
from argparse import SUPPRESS

import lal

import json

In [56]:
def do_bayestar_run(ra_input,dec_input):
    '''provide a ra/dec in radians in the range (-pi, pi), (-pi/2, pi/2)'''
    #formats ra/dec for file names
    ra_0=float('{:.2f}'.format(ra_input))
    dec_0=float('{:.2f}'.format(dec_input))

    #converts ra/dec to degrees for file parameter inputs
    ra_dg=np.rad2deg(float('{:.2f}'.format(ra_0)))
    dec_dg=np.rad2deg(float('{:.2f}'.format(dec_0)))

    print(ra_0,dec_0,"  ",ra_dg,dec_dg)
    
    os.system("lalapps_inspinj \
        `# Write output to inj.xml.` \
        -o inj_{}_{}_50mpc.xml \
        `# Mass injection.` \
        `# In this example, the masses are pinned to 1.3 and 1.4 Msun.` \
        --m-distr fixMasses --fixed-mass1 1.5 --fixed-mass2 1.8 \
        `# sky location injection.` \
        --l-distr fixed --longitude {} --latitude {} \
        `# Distance distribution: uniform in Euclidean volume.` \
        `# WARNING: distances are in kpc.` \
        --d-distr volume \
        --min-distance 50000 --max-distance 50000 \
        `# Coalescence time distribution: adjust time step, start, and stop` \
        `# polarization (psi)` \
        --polarization  0 \
        `# Inclination angle (theta_jn)` \
        --i-distr  fixed --fixed-inc 85.943 \
        `# phase (phase)` \
        --fixed-coa-phase 57.295 \
        `# time to control the number of injections.` \
        --time-step 7200 \
        --gps-start-time 1264079176 \
        --gps-end-time 1264069576 \
        `# Distance distribution: uniform in Euclidean volume.` \
        `# Write a table of CBC injections to inj.xml.` \
        --f-lower 20 --disable-spin \
        --waveform TaylorF2threePointFivePN".format(ra_0,dec_0,ra_dg,dec_dg))

    print("#1")

    os.system("bayestar-realize-coincs \
        `# Write output to coinc.xml.` \
        -o coinc_{}_{}_50mpc.xml \
        `# Use the injections and noise PSDs that we generated.` \
        inj_{}_{}_50mpc.xml --reference-psd correct_psd.xml \
        `# Specify which detectors are in science mode.` \
        --detector H1 L1 V1 \
        `# Optionally, add Gaussian noise (rather than zero noise).` \
        --measurement-error gaussian-noise \
        `# Optionally, adjust the detection threshold: single-detector` \
        `# SNR, network SNR, and minimum number of detectors above` \
        `# threshold to form a coincidence.` \
        --snr-threshold 4.0 \
        --net-snr-threshold 12.0 \
        --min-triggers 2 \
        `# Optionally, save triggers that were below the single-detector` \
        `# threshold.` \
        --keep-subthreshold".format(ra_0,dec_0,ra_0,dec_0))

    print("#2")

    os.system("bayestar-localize-coincs coinc_{}_{}_50mpc.xml".format(ra_0, dec_0))
    
    print("#3")
    if (os.path.exists('0.fits')):
        os.system("mv 0.fits skymap_{}_{}_50mpc.fits".format(ra_0,dec_0))

        print("#4")

        os.system("ligo-skymap-plot skymap_{}_{}_50mpc.fits -o skymap_{}_{}_50mpc.png --annotate --contour 90 95".format(ra_0,dec_0,ra_0,dec_0))

        print("#5")
    else:
        print('fits file not generated')

In [53]:
#get the index location for the cumulative sum of 
# ordered hpdata
def return_percentile_index(hpdata, percent):
    ordered_hpdata=hpdata.copy()
    sigma=np.flip(np.argsort(ordered_hpdata))
    ordered_hpdata=ordered_hpdata[sigma]

    prob=0
    index=0
    while (prob<percent):
        prob+=ordered_hpdata[index]
        index+=1

    index-=1

    loc=sigma[index]

    return loc
    

In [54]:
#specify your ra/dec
your_ra=0.4
your_dec=0.73

bay_ra=your_ra-np.pi
bay_dec=your_dec

In [57]:
do_bayestar_run(bay_ra, bay_dec)

-2.74 0.73    -156.99043586584557 41.8259190445501
#1
#2
#3
#4
#5


In [61]:
ls

coinc_-0.9_1.35_75mpc.xml    skymap_-0.9_1.35_75mpc.fits
inj_-0.9_1.35_75mpc.xml      skymap_-0.9_1.35_75mpc.png


In [62]:
skymap_list=['skymap_-0.9_1.35_75mpc.fits']

for fits_file in skymap_list:
    
    #reads in flattened fits file
    #e.g. get healpy probability/ sky location data
    hpdata, header = read_sky_map(fits_file)
    
    #set up co-ordinates based on hpdata
    #(e.g. for a given hpdata, get ra/dec coordinate array)
    npix=len(hpdata)
    nside=hp.npix2nside(npix)
    theta, phi= hp.pix2ang(nside, np.arange(npix))
    dec_hp = np.rad2deg(np.pi/2-theta)
    ra_hp = np.rad2deg(phi-2*np.pi)
    
    #get array-space index
    index=return_percentile_index(hpdata,0.925)  #put p% here

    #prints info about current fits_file p% index
    print(" ")
    print("name: ", fits_file)
    print("index: ",index)
    print("ra: ",ra_hp[index])
    print("dec: ", dec_hp[index])
    print(" ")

 
name:  skymap_-0.9_1.35_75mpc.fits
index:  545713
ra:  -51.46551724137933
dec:  78.05449794559584
 


In [65]:
print('RA|    true: ', 2.239, ' |  followup: ', np.deg2rad(-51.46551724137933)+np.pi) 

print('DEC|   true: ', 1.346, ' |  followup: ', np.deg2rad(78.05449794559584)) 

angdist = pyasl.getAngDist(274.130859375, -5.360645802724182,
                 277.88453063844923, 0.0)

print('angular distance: ',angdist, ' | in radians: ' ,angdist*np.pi/180)

RA|    true:  2.239  |  followup:  2.2433499264858145
DEC|   true:  1.346  |  followup:  1.3623079851417972
angular distance:  6.541056682141486  | in radians:  0.1141629756629451


In [13]:
-265.01220703125+3*180

274.130859375

In [ ]:
bby.gw.conversion.component_masses_to_mass_ratio(1.4,1.3)

bby.gw.conversion.component_masses_to_chirp_mass(1.4,1.3)

pyasl.getAngDist(r, d, ra_hp[max_index], dec_hp[max_index])

q=bby.gw.conversion.component_masses_to_chirp_mass(1.8012,1.5004)
r=bby.gw.conversion.component_masses_to_mass_ratio(1.8012,1.5004)
print('chirp: ', q, "| ratio: ",r)

In [12]:
pyasl.getAngDist(0, 0, 0, 90)

89.99999999999999

In [26]:
hpdata

array([5.93794553e-14, 4.81901925e-18, 7.58288455e-19, ...,
       4.74397435e-20, 5.99461340e-20, 2.07296221e-19])

In [30]:
dec_hp

array([ 89.97715732,  89.97715732,  89.97715732, ..., -89.97715732,
       -89.97715732, -89.97715732])

In [33]:
ra_hp

array([-315., -225., -135., ..., -225., -135.,  -45.])

In [59]:
os.system("ligo-skymap-plot skymap_-2.74_0.73_50mpc.fits -o skymap_-2.74_0.73_50mpc.png --annotate --contour 90 95")

0